### Курсовая работа

В данной работе будем предсказывать тип звезды по её основным признакам.
Датасет взят тут: https://www.kaggle.com/deepu1109/star-dataset

#### Описание датасета

 - **Temperature** - температура в кельвинах
 - **Luminosity** - светимость звезды в единицах светимости Солнца
 - **Radius** - радиус звезды расчитанный относительно радиуса Солнца
 - **Absolute magnitude** - абсолютная звездная величина
 - **Star colo**r - цвет здезды
 - **Spectral Class** - класс спектра
 - **Star type** - тип звезды (целевой признак который будем предсказывать)

Тип зведы включает в себя 6 классов, от 0 до 5

- 0 - Коричневый карлик
- 1 - Красный карлик
- 2 - Белый карлик
- 3 - Главная последовательность
- 4 - Супер гигант
- 5 - Гипер гигант

In [1]:
import pandas as pd
import dill
import numpy as np

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [2]:
#загрузим датасет

df = pd.read_csv("./data.csv")
df.head(3)

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color,Spectral Class
0,3068,0.0024,0.1700,16.12,0,Red,M
1,3042,0.0005,0.1542,16.60,0,Red,M
2,2600,0.0003,0.1020,18.70,0,Red,M


In [3]:
df.shape

(240, 7)

In [4]:
df["Star color"].unique()

array(['Red', 'Blue White', 'White', 'Yellowish White', 'Blue white',
       'Pale yellow orange', 'Blue', 'Blue-white', 'Whitish',
       'yellow-white', 'Orange', 'White-Yellow', 'white', 'Blue ',
       'yellowish', 'Yellowish', 'Orange-Red', 'Blue white ',
       'Blue-White'], dtype=object)

In [18]:
df["Spectral Class"].unique()

array(['M', 'B', 'A', 'F', 'O', 'K', 'G'], dtype=object)

Всего в нашем датасете 240 записей и 7 признаков среди которых один целевой.

Посмотрим есть ли пропуски в данных и какой тип имеют данные

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Temperature (K)         240 non-null    int64  
 1   Luminosity(L/Lo)        240 non-null    float64
 2   Radius(R/Ro)            240 non-null    float64
 3   Absolute magnitude(Mv)  240 non-null    float64
 4   Star type               240 non-null    int64  
 5   Star color              240 non-null    object 
 6   Spectral Class          240 non-null    object 
dtypes: float64(3), int64(2), object(2)
memory usage: 13.2+ KB


Разделим данные на тренировочные и на отложенную выборку для проверки, под отложенную выборку возьмем четверть всех данных

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df, df['Star type'], random_state=42, test_size=0.25)

In [7]:
X_train.shape

(180, 7)

In [8]:
X_test.shape

(60, 7)

Построим пайплайн.

- Категориальные признаки закодируем с помощью OneHotEncoding
- А к вещественным применим standardScaler

In [9]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]

class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [10]:
categorical_columns = ["Star color", "Spectral Class"]
continuous_columns = ["Temperature (K)", "Luminosity(L/Lo)", "Radius(R/Ro)", "Absolute magnitude(Mv)"]

In [11]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

Объединим все это в единый пайплайн

In [12]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

Обучим данные на модели случайного леса

In [13]:
pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

In [14]:
#обучим наш пайплайн
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Star color',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Star '
                                                                                         'color')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Star '
                                                                                 'color'))])),
                                                ('Spectral Class',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Spectral '
                                                                                         'Class')),
                              

In [15]:
y_predict = pipeline.predict(X_test)

y_predict

array([2, 0, 3, 4, 4, 5, 5, 2, 0, 2, 4, 2, 5, 4, 5, 5, 0, 1, 3, 0, 0, 1,
       5, 3, 5, 3, 1, 2, 5, 3, 3, 4, 2, 0, 1, 0, 1, 5, 1, 5, 4, 5, 4, 1,
       3, 0, 4, 3, 1, 0, 2, 0, 2, 1, 4, 2, 5, 3, 1, 1], dtype=int64)

Проверим качество модели

In [16]:
results = cross_val_score(estimator=pipeline, X=X_train, y=y_train, cv=10, n_jobs=-1)
print('model accuracy score: {:.3f}'.format(results.mean()))
print('model train accuracy score: {:.3f}'.format(pipeline.score(X_train, y_train)))
print('model test accuracy score: {:.3f}'.format(pipeline.score(X_test, y_test)))

model accuracy score: 1.000
model train accuracy score: 1.000
model test accuracy score: 1.000


Модель в 100% случаев предсказывает тип зведы верно

Сохраним модель для дальнейшего использования

In [17]:
with open("stars_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)